In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive

/content/drive


In [4]:
cd My\ Drive

/content/drive/My Drive


In [6]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [7]:
cd Text-Style-Transfer/

/content/drive/My Drive/Colab Notebooks/Text-Style-Transfer


In [9]:
!pip install scikit-learn==0.22

     |████████████████████████████████| 7.0MB 2.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [17]:
import json
import numpy as np
import collections
import logging
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
print(dir(nltk.corpus))

NameError: ignored

In [0]:
class Vocab:
    def __init__(self):
        self.vocab_size = 9203
        self.train_file_path = "update_dataset/train.txt"
        self.vocab_save_path = "update_dataset"
        self.predefined_word_index = {"<pad>": 0, "<sos>": 1, "<unk>": 2,}
        self.filter_sentiment_words = True
        self.filter_stopwords = True

    def create_vocab(self):
        index2word = dict()
        words = collections.Counter()
        word2index = self.predefined_word_index
        with open(self.train_file_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                if len(line) == 0:
                    continue
                words.update(line.split())
        words = words.most_common(self.vocab_size)
        logging.debug("collected {} most common words".format(self.vocab_size))
        emb_matrix = np.zeros(
            (self.vocab_size + len(self.predefined_word_index), 300))
        emb_matrix[:len(self.predefined_word_index), :] = np.random.rand(
            len(self.predefined_word_index), 300)
        w2v_model = KeyedVectors.load_word2vec_format(
            "word_embedding/word_embedding.txt").wv
        idx = 0
        for word, index in self.predefined_word_index.items():
            word2index[word] = index
            index2word[index] = word
            idx += 1
        for token in words:
            if token[0] in w2v_model:
                word2index[token[0]] = idx
                index2word[idx] = token[0]
                emb_matrix[idx, :] = w2v_model[token[0]]
                idx = idx + 1
        with open(self.vocab_save_path + '/word2index.json', 'w') as json_file:
            json.dump(word2index, json_file)
        with open(self.vocab_save_path + '/index2word.json', 'w') as json_file:
            json.dump(index2word, json_file)
        np.save("word_embedding/word_embedding.txt", emb_matrix)
        self._populate_word_blacklist(word2index)

    def _populate_word_blacklist(self, word_index):
        blacklisted_words = set()
        bow_filtered_vocab_indices = dict()
        blacklisted_words |= set(self.predefined_word_index.values())
        if self.filter_sentiment_words:
            blacklisted_words |= self._get_sentiment_words()
        if self.filter_stopwords:
            blacklisted_words |= self._get_stopwords()
        allowed_vocab = word_index.keys() - blacklisted_words
        i = 0
        for word in allowed_vocab:
            vocab_index = word_index[word]
            bow_filtered_vocab_indices[vocab_index] = i
            i += 1
        print("BoW_Size: " + str(len(allowed_vocab)))
        with open(self.vocab_save_path + '/bow.json', 'w') as json_file:
            json.dump(bow_filtered_vocab_indices, json_file)

    def _get_sentiment_words(self):
        with open(file="dataset/train.0",
                  mode='r', encoding='ISO-8859-1') as pos_sentiment_words_file,\
            open(file="dataset/train.0",
                 mode='r', encoding='ISO-8859-1') as neg_sentiment_words_file:
            pos_words = pos_sentiment_words_file.readlines()
            neg_words = neg_sentiment_words_file.readlines()
            words = pos_words + neg_words
        words = set(word.strip() for word in words)
        return words

    def _get_stopwords(self):
        nltk_stopwords = set(stopwords.words('english'))
        sklearn_stopwords = stop_words.ENGLISH_STOP_WORDS
        all_stopwords = set()
        all_stopwords |= spacy_stopwords
        all_stopwords |= nltk_stopwords
        all_stopwords |= sklearn_stopwords
        return all_stopwords

In [21]:
vocab = Vocab()
vocab.create_vocab()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


BoW_Size: 8886
